
### Zadanie prostsze: Analiza fake newsów na podstawie tytułów, 


Twoim zadaniem jest stworzenie modelu sieci neuronowej, który będzie klasyfikować tweety na podstawie tytułu. W danych mamy dostarczone tytuły artykułów oraz etykiety fake news (1) albo nie (0). 
Kroki, które musisz podjąć:

1. Przygotowanie danych: Dane zostały wczytane z pliku CSV. Teksty znajdują się w kolumnach title, a etykiety w kolumnie "class". Teksty oraz etykiety są przygotowane do dalszej obróbki.

2. Tokenizacja tekstu: Teksty powinny być tokenizowane za pomocą Tokenizer, a następnie przekształcane na sekwencje liczb. Jest to niezbędne, aby można było używać ich jako danych wejściowych do modelu sieci neuronowej.

3. Zbuduj model sieci neuronowej: Model sieci rekurencyjnej zdefiniuj w kodzie i przykładowo niech składa się z następujących warstw:

* Warstwa wejściowa: Przyjmuje sekwencje tokenów o długości określonej przez max_sequence_length.
* Warstwa Embedding: Mapuje tokeny na wektory o wymiarze embed_dim (100 w tym przypadku), co pozwala na reprezentację słów jako gęstych wektorów cech.
* Warstwa MultiHeadAttention: Implementuje mechanizm uwagi, który pozwala modelowi skupić się na różnych częściach sekwencji wejściowej, co jest szczególnie przydatne w przetwarzaniu języka naturalnego.
* GlobalAveragePooling1D: Agreguje informacje z różnych części sekwencji, redukując wymiarowość danych i przygotowując je do procesu klasyfikacji.
* Warstwa Dense (wyjściowa): Z funkcją aktywacji softmax, generuje dystrybucję prawdopodobieństwa dla dwóch możliwych klas sentymentu, umożliwiając klasyfikację.


4. Kompilacja modelu: Model jest kompilowany z optymalizatorem Adam, funkcją straty "categorical_crossentropy" (odpowiednia do klasyfikacji wieloklasowej) oraz metryką "accuracy".

5. Trenowanie modelu: Model jest trenowany na danych tweetów przez 5 epok, z wsadem o rozmiarze 32. Wynik trenowania jest oceniany na zbiorze walidacyjnym.

Twoim zadaniem jest uzupełnienie kodu i dostosowanie parametrów modelu oraz trenowania, aby uzyskać jak najlepsze wyniki w zadaniu analizy fake newsów na podstawie tytułów. Możesz eksperymentować z parametrami, takimi jak liczba epok trenowania, rozmiar wsadu (batch size).


In [2]:
import pandas as pd 
import numpy as np 

fake=pd.read_csv("/kaggle/input/fake-news-detection/fake.csv")
true=pd.read_csv("/kaggle/input/fake-news-detection/true.csv")

fake['class']=0
fake.head()

true['class']=1
true.head()


data=pd.concat([fake,true],axis=0)

In [3]:
X = data['title'].to_numpy()
y = data['class'].to_numpy()

In [12]:
fake


,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
...,...,...,...,...,...
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016",0
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016",0
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016",0
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",0


In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
# Tokenizacja tekstu
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)
word_index = tokenizer.word_index
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(sequences, y, test_size=0.3, random_state=42)

max_sequence_length = max(len(seq) for seq in X)
X_train = pad_sequences(X_train, maxlen=max_sequence_length)
X_val = pad_sequences(X_val, maxlen=max_sequence_length)


2024-02-22 17:47:12.012854: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-22 17:47:12.013006: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-22 17:47:12.197799: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [11]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, MultiHeadAttention, GlobalAveragePooling1D, Dense

embed_dim = 32  # Wymiar wektora embeddingowego
num_heads = 2  # Liczba "głowic" w warstwie Multi-Head Attention
vocab_size = 10000  # Rozmiar słownika

inputs = Input(shape=(max_sequence_length,))
embedding_layer = Embedding(input_dim=vocab_size, output_dim=embed_dim)(inputs)
attention_output = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)(embedding_layer, embedding_layer)
global_average = GlobalAveragePooling1D()(attention_output)
outputs = Dense(1, activation='sigmoid')(global_average)

model = Model(inputs=inputs, outputs=outputs)

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=320, validation_split=0.1, verbose=1)




Epoch 1/10
89/89 [==============================] - 12s 109ms/step - loss: 0.6862 - accuracy: 0.5541 - val_loss: 0.6458 - val_accuracy: 0.5409
Epoch 2/10
89/89 [==============================] - 7s 75ms/step - loss: 0.3276 - accuracy: 0.8975 - val_loss: 0.1515 - val_accuracy: 0.9415
Epoch 3/10
89/89 [==============================] - 5s 56ms/step - loss: 0.1204 - accuracy: 0.9562 - val_loss: 0.1241 - val_accuracy: 0.9513
Epoch 4/10
89/89 [==============================] - 5s 54ms/step - loss: 0.0924 - accuracy: 0.9679 - val_loss: 0.1092 - val_accuracy: 0.9590
Epoch 5/10
89/89 [==============================] - 5s 51ms/step - loss: 0.0706 - accuracy: 0.9762 - val_loss: 0.1012 - val_accuracy: 0.9640
Epoch 6/10
89/89 [==============================] - 5s 52ms/step - loss: 0.0566 - accuracy: 0.9812 - val_loss: 0.0977 - val_accuracy: 0.9650
Epoch 7/10
89/89 [==============================] - 4s 43ms/step - loss: 0.0424 - accuracy: 0.9859 - val_loss: 0.0946 - val_accuracy: 0.9669
Epoch 8/10



### Zadanie prostsze: Analiza fake newsów na podstawie tytułów z PositionalEncoding


Twoim zadaniem jest stworzenie modelu sieci neuronowej, który będzie klasyfikować na podstawie tytułu. W danych mamy dostarczone tytuły artykułów oraz etykiety fake news (1) albo nie (0). 
Kroki, które musisz podjąć:

1. Przygotowanie danych: Dane zostały wczytane z pliku CSV. Teksty znajdują się w kolumnach title, a etykiety w kolumnie "class". Teksty oraz etykiety są przygotowane do dalszej obróbki.

2. Tokenizacja tekstu: Teksty powinny być tokenizowane za pomocą Tokenizer, a następnie przekształcane na sekwencje liczb. Jest to niezbędne, aby można było używać ich jako danych wejściowych do modelu sieci neuronowej.

3. Zbuduj model sieci neuronowej: Model sieci rekurencyjnej zdefiniuj w kodzie i przykładowo niech składa się z następujących warstw:

* Warstwa wejściowa: Przyjmuje sekwencje tokenów o długości określonej przez max_sequence_length.
* Warstwa Embedding: Mapuje tokeny na wektory o wymiarze embed_dim (100 w tym przypadku), co pozwala na reprezentację słów jako gęstych wektorów cech.
* Warstwa PositionalEncoding(position, embed_dim)(embedding_layer), która mapuje sekwencje na odpowiednie pozycje. 
* Warstwa MultiHeadAttention: Implementuje mechanizm uwagi, który pozwala modelowi skupić się na różnych częściach sekwencji wejściowej, co jest szczególnie przydatne w przetwarzaniu języka naturalnego.
* GlobalAveragePooling1D: Agreguje informacje z różnych części sekwencji, redukując wymiarowość danych i przygotowując je do procesu klasyfikacji.
* Warstwa Dense (wyjściowa): Z funkcją aktywacji softmax, generuje dystrybucję prawdopodobieństwa dla dwóch możliwych klas sentymentu, umożliwiając klasyfikację.


4. Kompilacja modelu: Model jest kompilowany z optymalizatorem Adam, funkcją straty "categorical_crossentropy" (odpowiednia do klasyfikacji wieloklasowej) oraz metryką "accuracy".

5. Trenowanie modelu: Model jest trenowany na danych tweetów przez 5 epok, z wsadem o rozmiarze 32. Wynik trenowania jest oceniany na zbiorze walidacyjnym.

Twoim zadaniem jest uzupełnienie kodu i dostosowanie parametrów modelu oraz trenowania, aby uzyskać jak najlepsze wyniki w zadaniu analizy fake newsów na podstawie tytułów. Możesz eksperymentować z parametrami, takimi jak liczba epok trenowania, rozmiar wsadu (batch size).


In [38]:
X = data['title'].to_numpy()
y = data['class'].to_numpy()

In [39]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
# Tokenizacja tekstu
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)
word_index = tokenizer.word_index
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(sequences, y, test_size=0.3, random_state=42)

#max_sequence_length = 10000
max_sequence_length = max(len(seq) for seq in X)
X_train = pad_sequences(X_train, maxlen=max_sequence_length)
X_val = pad_sequences(X_val, maxlen=max_sequence_length)

In [21]:
max_sequence_length

286

In [20]:
from tensorflow.keras.layers import Layer, Embedding, Dense, Dropout, LayerNormalization
from tensorflow.keras.models import Model
from tensorflow.keras import Input

class PositionalEncoding(Layer):
   def __init__(self, position, d_model):
       super(PositionalEncoding, self).__init__()
       self.pos_encoding = self.positional_encoding(position, d_model)

   def get_angles(self, position, i, d_model):
       angles = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
       return position * angles

   def positional_encoding(self, position, d_model):
       angle_rads = self.get_angles(np.arange(position)[:, np.newaxis],
                                    np.arange(d_model)[np.newaxis, :],
                                    d_model)
       # Aplikuj sinus do indeksów parzystych w macierzy; 2i
       angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
       # Aplikuj cosinus do indeksów nieparzystych w macierzy; 2i+1
       angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
       pos_encoding = angle_rads[np.newaxis, ...]
       return tf.cast(pos_encoding, dtype=tf.float32)

   def call(self, inputs):
       return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]



In [27]:
from keras import layers
class TokenAndPositionEmbedding(layers.Layer):
   def __init__(self, maxlen, vocab_size, embed_dim):
       super().__init__()
       self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
       self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

   def call(self, x):
       # Obliczanie długości sekwencji
       maxlen = tf.shape(x)[-1]
       # Generowanie pozycji dla każdego tokenu w sekwencji
       positions = tf.range(start=0, limit=maxlen, delta=1)
       # Uzyskanie pozycyjnego embedding
       positions = self.pos_emb(positions)
       # Uzyskanie tlenowego embedding
       x = self.token_emb(x)
       # Dodanie pozycyjnego embedding do tlenowego embedding
       return x + positions


In [30]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, MultiHeadAttention, GlobalAveragePooling1D, Dense

embed_dim = 32  # Wymiar wektora embeddingowego
num_heads = 2  # Liczba "głowic" w warstwie Multi-Head Attention
vocab_size = 10000  # Rozmiar słownika

inputs = Input(shape=(max_sequence_length,))
#embedding_layer = Embedding(input_dim=vocab_size, output_dim=embed_dim)(inputs)
positional_layer=TokenAndPositionEmbedding(max_sequence_length,vocab_size, embed_dim)(inputs),
attention_output = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)(positional_layer, positional_layer)
global_average = GlobalAveragePooling1D()(attention_output)
outputs = Dense(1, activation='sigmoid')(global_average)

model = Model(inputs=inputs, outputs=outputs)

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=320, validation_split=0.1, verbose=1)


TypeError: Exception encountered when calling layer "multi_head_attention_8" (type MultiHeadAttention).

Dimension value must be integer or None or have an __index__ method, got value '<tf.Tensor 'Placeholder:0' shape=(None, 286, 32) dtype=float32>' with type '<class 'tensorflow.python.framework.ops.SymbolicTensor'>'

Call arguments received by layer "multi_head_attention_8" (type MultiHeadAttention):
  • query=('tf.Tensor(shape=(None, 286, 32), dtype=float32)',)
  • value=('tf.Tensor(shape=(None, 286, 32), dtype=float32)',)
  • key=None
  • attention_mask=None
  • return_attention_scores=False
  • training=None
  • use_causal_mask=False

In [40]:
inputs = Input(shape=(max_sequence_length,))
embedding_layer = Embedding(input_dim=vocab_size, output_dim=embed_dim)(inputs)
pos_encoding = PositionalEncoding(max_sequence_length, embed_dim)(embedding_layer)
attention_output = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)(pos_encoding, pos_encoding)
global_average = GlobalAveragePooling1D()(attention_output)
outputs = Dense(1, activation='sigmoid')(global_average)

model = Model(inputs=inputs, outputs=outputs)

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=320, validation_split=0.1, verbose=1)

Epoch 1/10
89/89 [==============================] - 10s 96ms/step - loss: 0.6920 - accuracy: 0.5235 - val_loss: 0.6849 - val_accuracy: 0.9055
Epoch 2/10
89/89 [==============================] - 7s 81ms/step - loss: 0.2904 - accuracy: 0.9070 - val_loss: 0.1161 - val_accuracy: 0.9529
Epoch 3/10
89/89 [==============================] - 6s 65ms/step - loss: 0.0786 - accuracy: 0.9711 - val_loss: 0.0914 - val_accuracy: 0.9640
Epoch 4/10
89/89 [==============================] - 4s 49ms/step - loss: 0.0475 - accuracy: 0.9841 - val_loss: 0.1173 - val_accuracy: 0.9574
Epoch 5/10
89/89 [==============================] - 5s 52ms/step - loss: 0.0314 - accuracy: 0.9900 - val_loss: 0.0886 - val_accuracy: 0.9685
Epoch 6/10
89/89 [==============================] - 4s 45ms/step - loss: 0.0196 - accuracy: 0.9939 - val_loss: 0.1002 - val_accuracy: 0.9675
Epoch 7/10
89/89 [==============================] - 4s 42ms/step - loss: 0.0148 - accuracy: 0.9957 - val_loss: 0.1279 - val_accuracy: 0.9634
Epoch 8/10
8

# Zadanie: Analiza 16 typów osobowości na podstawie Tweetów
Twoim zadaniem jest stworzenie modelu sieci neuronowej, który będzie klasyfikować tweety na podstawie osobowości osób piszących. W danych mamy dostarczone tweety oraz etykiety osobowości (np. INTP, ENTJ).

## Kroki, które musisz podjąć:

1. Przygotowanie danych: Dane zostały wczytane z pliku CSV. Teksty tweetów znajdują się w kolumnach tweet_1, tweet_2, itp., a etykiety sentymentu w kolumnie "sentiment". Teksty tweetów oraz etykiety sentymentu są przygotowane do dalszej obróbki.

2. Tokenizacja tekstu: Teksty tweetów są tokenizowane za pomocą Tokenizer, a następnie przekształcane na sekwencje liczb. Jest to niezbędne, aby można było używać ich jako danych wejściowych do modelu sieci neuronowej.

3. Przygotuj model sieci neuronowej złożony z: 

* Warstwa wejściowa: Przyjmuje sekwencje tokenów o długości określonej przez max_sequence_length.
* Warstwa Embedding: Mapuje tokeny na wektory o wymiarze embed_dim (100 w tym przypadku), co pozwala na reprezentację słów jako gęstych wektorów cech.
* Warstwa MultiHeadAttention: Implementuje mechanizm uwagi, który pozwala modelowi skupić się na różnych częściach sekwencji wejściowej, co jest szczególnie przydatne w przetwarzaniu języka naturalnego.
* GlobalAveragePooling1D: Agreguje informacje z różnych części sekwencji, redukując wymiarowość danych i przygotowując je do procesu klasyfikacji.
* Warstwa Dense (wyjściowa): Z funkcją aktywacji softmax, generuje dystrybucję prawdopodobieństwa dla dwóch możliwych klas sentymentu, umożliwiając klasyfikację.

4. Trenowanie modelu: Model jest trenowany na danych tweetów przez 5 epok, z wsadem o rozmiarze 32. Wynik trenowania jest oceniany na zbiorze walidacyjnym.

Twoim zadaniem jest uzupełnienie kodu i dostosowanie parametrów modelu oraz trenowania, aby uzyskać jak najlepsze wyniki w zadaniu analizy 16 typów osobowości na podstawie tweetów. Możesz eksperymentować z parametrami, takimi jak liczba epok trenowania, rozmiar wsadu (batch size), wymiar embeddingu czy rozmiar warstwy LSTM, aby dostosować model do danego zadania analizy sentymentu tekst

In [2]:
import pandas as pd 

df = pd.read_csv('/kaggle/input/filtered-eng-twitter/languages_eng_cleaned.csv', index_col = 0)
df.head()

,id,tweet_1,tweet_2,tweet_3,tweet_4,tweet_5,tweet_6,tweet_7,tweet_8,tweet_9,...,tweet_193,tweet_194,tweet_195,tweet_196,tweet_197,tweet_198,tweet_199,tweet_200,mbti_personality,detected_language
1,<NUMBER>,"PLEASE VOTE, VOTE, VOTE FOR AMYBETH! thanks! i...",RT <USER>: Look at this cutie! Thank you for t...,"'kelangan talaga lumipat ng bahay, pero di ka ...",forgiveness and justice.\nforgiveness with jus...,hirap maging babae no? <TAG>PamilyaKoPagkabuwag,"eh damang-dama ko yung pagod ni luz, yung pago...","oo nga no? makes you think, what's your deal-b...",hay nako si apol timing fail talaga.\n<TAG>Pam...,RT <USER>: Halimaw si Sylvia Sanchez. In every...,...,RT <USER>: <USER> <URL> \nIn all of these scen...,RT <USER>: Who can pinpoint the exact moment A...,"RT <USER>: Dear Gilbert,\n\nI'm throwing the r...",RT <USER>: (ANNE NATION)\n\nwe are everyone \n...,RT <USER>: (what is love)\n\nloving is learnin...,RT <USER>: This Anne Nation. \n\nAdded active ...,RT <USER>: To fail means we’ve tried. To be hu...,<USER> hahahahahahaha! tingnan natin! mahal ko...,infp,en
3,<NUMBER>,RT <USER>: 🤷‍♀️\n<TAG>EXOLSelcaDay \n<USER> <URL>,RT <USER>: when is this from??? 😭😭😭 <URL>,"RT <USER>: since we're talking about suhø, a f...",I am supporting this fundraising page <URL> an...,RT <USER>: Sun and moon outfits <URL>,<USER> that looks like porridge AND TO DRAIN I...,"RT <USER>: Au Revoir, Paris (세훈)\n\n👉🏻<URL>\n\...",RT <USER>: this is definitely one of my fav se...,RT <USER>: oh to be watching the rain and list...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,infp,en
5,<NUMBER>,RT <USER>: The media are just feeding fear ove...,RT <USER>: How my mother feels about these che...,"RT <USER>: I know now, as an adult, it’s my re...","RT <USER>: In the right now, I know that you n...",RT <USER>: I grew up and still have moments of...,RT <USER>: I also never had space for my menta...,"RT <USER>: I grew up “tip toeing” , watching t...",RT <USER>: Let's discuss UK Wig Makers 😭🥺 <URL>,RT <USER>: When Nicole spilled the tea about P...,...,RT <USER>: Beyoncé's <NUMBER>/<NUMBER> video i...,RT <USER>: <TAG>InternationalWomensDay began o...,RT <USER>: It’s fascinating how our love langu...,Bread <URL>,Queen <URL>,And the land is dark <URL>,Couple days ago <URL>,NaN,infp,en
6,<NUMBER>,RT <USER>: Are y'all washing your hands at hom...,<USER> <URL>,"<USER> not ""bun in the front"". straight to the...","RT <USER>: Doug Collins, who has self-quaranti...",<USER> <USER> Imagine if she challenged you to...,<USER> They really do!,<USER> Stepping that ground game up and not ha...,RT <USER>: Folks should spend more time on doo...,<USER> <USER> *The real source of the breakup ...,...,<USER> being blocked is a blessing lol,<USER> <USER> it makes no sense.,<USER> 😭,<USER> yes <URL>,RT <USER>: If Bloomberg wanted to waste <NUMBE...,<USER> mugs really were saying they'd look bey...,Please inform all the ppl who were saying they...,<USER> 😭😭,infp,en
8,<NUMBER>,RT <USER>: <TAG>Supergirl really missed the ma...,RT <USER>: Wild how most of the media response...,RT <USER>: Let it be known that these are the ...,RT <USER>: The ultimate ghost Pokemon got ghos...,RT <USER>: Dear ableds: Panic buying is not go...,"<USER> exactly, and corporate world is really ...",I would report him to HR and my employer will ...,<USER> <USER> <USER> <USER> are you guys even ...,<USER> i can stay home for a long period of ti...,...,"oh dear, youve only got <NUMBER> followers so ...",<USER> Katie McGrath,"<USER> <USER> ""except like if they started hav...",TRIED TO MAKE BANANA PANCAKE BUT IT FAILED 🙃 <...,RT <USER>: Never going to an anime convention ...,"RT <USER>: ""I'm gonna be a SOLDIER. The best o...",NaN,NaN,infp,en


In [3]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# Konwersja tweetów na pojedynczy tekst dla każdego użytkownika
df['all_tweets'] = df.apply(lambda row: ' '.join([str(tweet) for tweet in row[1:201]]), axis=1)


2024-02-22 18:56:35.929125: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-22 18:56:35.929252: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-22 18:56:36.100248: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import numpy as np

# Przygotowanie etykiet (MBTI)
labels = pd.get_dummies(df['mbti_personality']).values


In [6]:
X=df['all_tweets'].to_numpy()
y=labels

In [7]:
# Tokenizacja tekstu
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)
word_index = tokenizer.word_index
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(sequences, y, test_size=0.3, random_state=42)

max_sequence_length = 5000
#max_sequence_length = max(len(seq) for seq in X)
X_train = pad_sequences(X_train, maxlen=max_sequence_length)
X_val = pad_sequences(X_val, maxlen=max_sequence_length)

In [57]:
max(len(seq) for seq in X)

26634

In [ ]:
from tensorflow.keras.layers import Input, Embedding, MultiHeadAttention, GlobalAveragePooling1D, Dense
from tensorflow.keras.models import Model
import tensorflow as tf

embed_dim = 32  # Wymiar wektora embeddingowego
num_heads = 2  # Liczba "głowic" w warstwie Multi-Head Attention
vocab_size = 10000  # Rozmiar słownika

inputs = Input(shape=(max_sequence_length,))
embedding_layer = Embedding(input_dim=vocab_size, output_dim=embed_dim)(inputs)
#pos_encoding = PositionalEncoding(max_sequence_length, embed_dim)(embedding_layer)
attention_output = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)(embedding_layer, embedding_layer)
global_average = GlobalAveragePooling1D()(attention_output)
outputs = Dense(16, activation='softmax')(global_average)

model = Model(inputs=inputs, outputs=outputs)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=4, validation_split=0.1, verbose=1, validation_data=(X_val,y_val))

Epoch 1/10
1504/1504 [==============================] - 277s 183ms/step - loss: 1.8563 - accuracy: 0.0906 - val_loss: 1.7781 - val_accuracy: 0.0962
Epoch 2/10
1504/1504 [==============================] - 226s 150ms/step - loss: 1.8442 - accuracy: 0.1046 - val_loss: 1.8625 - val_accuracy: 0.3579
Epoch 3/10
1504/1504 [==============================] - 204s 136ms/step - loss: 1.9419 - accuracy: 0.1072 - val_loss: 1.9869 - val_accuracy: 0.0962
Epoch 4/10
 855/1504 [================>.............] - ETA: 1:13 - loss: 2.0643 - accuracy: 0.1053